# Obter dados no drive

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from google.colab import drive
import pandas as pd
import random
import os
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
from keras import layers, models, Input, regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
drive.mount('/content/drive')

# Carregamento de dados - Sem Data Augmentation


In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
with tf.device('/gpu:0'):
    # Lista para armazenar imagens e máscaras pré-processadas
    images_processed = []
    masks_processed = []

    # Carregar e pré-processar todas as imagens e máscaras
    for img_path, mask_path in zip(images, masks):
        img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
        images_processed.append(img)
        masks_processed.append(mask)

    # Converter para arrays numpy
    images_processed = np.array(images_processed)
    masks_processed = np.array(masks_processed)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.3, random_state=42)

# Definição de parâmetros e modelo

In [ ]:
# Função para calcular a sigmoide e converter para 0 ou 1 o output
class ThresholdLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.where(inputs < 0.5, 0.0, 1.0)

In [ ]:
# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

In [ ]:
def Unet(input_shape, dropout_rate):
    num_filters=(16, 32, 64, 128)
    kernel_size=3
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder (contraction path)
    conv1 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same')(conv1)
    drop1 = layers.Dropout(dropout_rate)(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(drop1)

    conv2 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same')(conv2)
    drop2 = layers.Dropout(dropout_rate)(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(drop2)

    conv3 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same')(conv3)
    drop3 = layers.Dropout(dropout_rate)(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(drop3)

    # Bottleneck
    conv4 = layers.Conv2D(num_filters[3], kernel_size, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(num_filters[3], kernel_size, activation='relu', padding='same')(conv4)
    drop4 = layers.Dropout(dropout_rate)(conv4)

    # Decoder (expansion path)
    up5 = layers.Conv2DTranspose(num_filters[2], (2, 2), strides=(2, 2), padding='same')(drop4)
    merge5 = layers.concatenate([conv3, up5], axis=3)
    conv5 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same')(merge5)
    conv5 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same')(conv5)

    up6 = layers.Conv2DTranspose(num_filters[1], (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = layers.concatenate([conv2, up6], axis=3)
    conv6 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2DTranspose(num_filters[0], (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = layers.concatenate([conv1, up7], axis=3)
    conv7 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same')(conv7)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv7)  # Saída com um canal (máscara binária)

    threshold_output = ThresholdLayer()(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Testes com dropout de 0.5

In [ ]:
def train_model():
  model = Unet(input_shape=(256, 256, 3), dropout_rate=0.5)

  with tf.device('/gpu:0'):
      # Compilar o modelo
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=[custom_accuracy])

      # Calcula o tempo de treino
      start_time = time.time()

      # Definições
      max_epochs = 50
      batch_size = 16

      # Treinar o modelo
      H = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=max_epochs, batch_size=batch_size)

      # Obter métricas de precisão e perda do histórico de treinamento
      acc = H.history['custom_accuracy']
      val_acc = H.history['val_custom_accuracy']
      loss = H.history['loss']
      val_loss = H.history['val_loss']

      # Número de épocas
      epochs = range(1, len(acc) + 1)

      # Calcula o tempo de inferência
      start_time = time.time()

      # Prever máscaras usando o modelo
      predicted_masks = model.predict(X_val)

      # Lista para armazenar os scores de IoU
      iou_scores = []
      correct_predictions = 0
      iou_threshold = 0.5
      for mask, result in zip(y_val, predicted_masks):
          intersection = np.logical_and(mask, result)
          union = np.logical_or(mask, result)
          iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
          iou_scores.append(iou_score)
          if iou_score >= iou_threshold:
              correct_predictions += 1

      iou_mean = np.mean(iou_scores)

      total_predictions = len(iou_scores)
      covr = correct_predictions / total_predictions if total_predictions > 0 else 0

  return [model,iou_mean,covr]

In [ ]:
# Criar modelo U-Net
total_loss = 0
total_acc = 0
total_iou = 0
total_covr = 0
for i in range(4):

  print("Início do teste ", i+1)
  model, iou_mean, covr = train_model()

  results = model.evaluate(X_val, y_val)

  total_loss += results[0]
  total_acc += results[1]
  total_iou += iou_mean
  total_covr += covr

print("A perda média foi de ", (total_loss/4))
print("A acurácia média foi de ", (total_acc/4))
print("O iou médio foi de ", (total_iou/4))
print("O covr médio foi de ", (total_covr/4))